# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [ ]:
# Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [26]:
data_quantity = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
data_quantity

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [27]:
product_customer_matrix = pd.pivot_table(data_quantity, values='Quantity', index=['ProductName'], columns=['CustomerID'], fill_value=0)
product_customer_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [29]:
distance_matrix = pdist(product_customer_matrix.T, metric='euclidean')
similarity_matrix = pd.DataFrame(squareform(distance_matrix), index=product_customer_matrix.columns, columns=product_customer_matrix.columns)

similarity_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [31]:
target_customer_id = 33  
target_customer_purchases = table[target_customer_id]

similar_customers = similarity_matrix[target_customer_id].sort_values(ascending=False).index[1:]

top_similar_customers = similar_customers[:5]

top_similar_customers



Index([97324, 95017, 92492, 91777, 92525], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [35]:
selected_records = data_quantity[data_quantity['CustomerID'].isin(top_similar_customers)]
selected_records

,CustomerID,ProductName,Quantity
59478,91777,"Appetizer - Mini Egg Roll, Shrimp",24
59479,91777,Banana - Leaves,24
59480,91777,Banana Turning,24
59481,91777,Bandage - Fexible 1x3,24
59482,91777,Bandage - Flexible Neon,24
...,...,...,...
62914,97324,Wine - Redchard Merritt,25
62915,97324,Wine - Vineland Estate Semi - Dry,25
62916,97324,"Wine - White, Colubia Cresh",25
62917,97324,Wonton Wrappers,25


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [36]:

aggregated_records = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
ranked_records = aggregated_records.sort_values(by='Quantity', ascending=False)

ranked_records


,ProductName,Quantity
228,"Turnip - White, Organic",146
22,"Beef - Tenderlion, Center Cut",122
243,Whmis - Spray Bottle Trigger,98
162,Pastry - Butterscotch Baked,98
66,Clam Nectar,98
...,...,...
113,Ice Cream Bar - Oreo Cone,24
111,Ice Cream Bar - Drumstick,24
110,Hot Chocolate - Individual,24
107,Halibut - Fletches,24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [38]:
chosen_customer_id = 33 
customer_purchases = product_customer_matrix[chosen_customer_id]

merged_data = pd.merge(ranked_records, product_customer_matrix, on='ProductName')

not_purchased_products = merged_data[merged_data[chosen_customer_id] == 0]

top_recommendations = not_purchased_products.head(5)

top_recommendations[['ProductName', 'Quantity']]


,ProductName,Quantity
0,"Turnip - White, Organic",146
1,"Beef - Tenderlion, Center Cut",122
2,Whmis - Spray Bottle Trigger,98
3,Pastry - Butterscotch Baked,98
4,Clam Nectar,98


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [39]:
recommendations_dict = {}

unique_customer_ids = data_quantity['CustomerID'].unique()

for customer_id in unique_customer_ids:
    similarities = similarity_matrix[customer_id].sort_values(ascending=False).index[1:]
    top_similar_customers = similarities[:5]

    selected_records = data_quantity[data_quantity['CustomerID'].isin(top_similar_customers)]

    aggregated_records = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
    ranked_records = aggregated_records.sort_values(by='Quantity', ascending=False)

    customer_purchases = product_customer_matrix[customer_id]
    merged_data = pd.merge(ranked_records, product_customer_matrix, on='ProductName')
    not_purchased_products = merged_data[merged_data[customer_id] == 0]
    top_recommendations = not_purchased_products.head(5)

    recommendations_dict[customer_id] = top_recommendations[['ProductName', 'Quantity']]

for customer_id, recommendations in recommendations_dict.items():
    print(f"CustomerID: {customer_id}\nRecommendations:\n{recommendations}\n{'='*40}\n")


CustomerID: 33
Recommendations:
                     ProductName  Quantity
0        Turnip - White, Organic       146
1  Beef - Tenderlion, Center Cut       122
2   Whmis - Spray Bottle Trigger        98
3    Pastry - Butterscotch Baked        98
4                    Clam Nectar        98

CustomerID: 200
Recommendations:
                     ProductName  Quantity
1   Whmis - Spray Bottle Trigger       124
2  Beef - Tenderlion, Center Cut       122
3           Bread - Hot Dog Buns       100
4  Pastry - Raisin Muffin - Mini        99
5    Pastry - Butterscotch Baked        98

CustomerID: 264
Recommendations:
                    ProductName  Quantity
0       Turnip - White, Organic       146
2  Whmis - Spray Bottle Trigger        98
4                   Clam Nectar        98
6           Squid U5 - Thailand        97
7          Coffee - Irish Cream        75

CustomerID: 356
Recommendations:
                     ProductName  Quantity
0        Turnip - White, Organic       146
1  Beef - Te

CustomerID: 31113
Recommendations:
                  ProductName  Quantity
0           Dc - Frozen Momji        97
1         Squid U5 - Thailand        97
2               Bagel - Plain        97
3     Olive - Spread Tapenade        97
4  Wine - Chablis 2003 Champs        96

CustomerID: 31323
Recommendations:
                        ProductName  Quantity
0        Spice - Peppercorn Melange        98
2                 Olives - Kalamata        97
3  Wine - White, Schroder And Schyl        97
4           Rum - Mount Gay Eclipes        97
5                     Bagel - Plain        97

CustomerID: 31373
Recommendations:
                     ProductName  Quantity
0        Turnip - White, Organic       122
1  Beef - Tenderlion, Center Cut       122
2        Bandage - Flexible Neon        99
3                   Tahini Paste        99
4      Coconut - Shredded, Sweet        99

CustomerID: 31409
Recommendations:
                     ProductName  Quantity
0        Turnip - White, Organic       1

CustomerID: 59379
Recommendations:
                     ProductName  Quantity
0                Cream Of Tartar       123
1  Ecolab - Lime - A - Way 4/4 L       100
2               Vinegar - Sherry        97
3                 Shrimp - 31/40        75
4           Bread - Hot Dog Buns        75

CustomerID: 59399
Recommendations:
                    ProductName  Quantity
0       Turnip - White, Organic       123
1          Bread - Hot Dog Buns       100
2           Beef - Inside Round       100
3                 Bagel - Plain       100
4  Whmis - Spray Bottle Trigger       100

CustomerID: 59626
Recommendations:
                     ProductName  Quantity
0        Turnip - White, Organic       123
1   Whmis - Spray Bottle Trigger       100
2           Bread - Hot Dog Buns       100
3  Pastry - Raisin Muffin - Mini        99
4                    Clam Nectar        98

CustomerID: 59703
Recommendations:
                ProductName  Quantity
0               Scampi Tail       123
1      Pecan 

CustomerID: 88769
Recommendations:
                 ProductName  Quantity
0   Cheese - Taleggio D.o.p.       118
1           Scallops - 10/20        96
2            Garlic - Peeled        96
3  Placemat - Scallop, White        96
4            Skirt - 29 Foot        96

CustomerID: 88855
Recommendations:
                  ProductName  Quantity
0  Soup V8 Roasted Red Pepper       120
1     Bandage - Flexible Neon        99
2                Onion Powder        98
3           Juice - V8 Splash        97
4           Olives - Kalamata        97

CustomerID: 89110
Recommendations:
                    ProductName  Quantity
0               Fondant - Icing       100
1    Spice - Peppercorn Melange        98
2  Soup - Campbells Bean Medley        96
4                   Scampi Tail        75
5            Cheese - Camembert        75

CustomerID: 89181
Recommendations:
                  ProductName  Quantity
0         Squid U5 - Thailand       121
1                Onion Powder        98
2         S

In [ ]:
## Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [41]:

result_columns = ['CustomerID'] + [f'Recommendation_{i+1}' for i in range(5)]
result_data = []

for customer_id, recommendations in recommendations_dict.items():
    customer_row = [customer_id] + recommendations['ProductName'].tolist()[:5]
    result_data.append(customer_row)

result_df = pd.DataFrame(result_data, columns=result_columns)

result_df


,CustomerID,Recommendation_1,Recommendation_2,Recommendation_3,Recommendation_4,Recommendation_5
0,33,"Turnip - White, Organic","Beef - Tenderlion, Center Cut",Whmis - Spray Bottle Trigger,Pastry - Butterscotch Baked,Clam Nectar
1,200,Whmis - Spray Bottle Trigger,"Beef - Tenderlion, Center Cut",Bread - Hot Dog Buns,Pastry - Raisin Muffin - Mini,Pastry - Butterscotch Baked
2,264,"Turnip - White, Organic",Whmis - Spray Bottle Trigger,Clam Nectar,Squid U5 - Thailand,Coffee - Irish Cream
3,356,"Turnip - White, Organic","Beef - Tenderlion, Center Cut",Whmis - Spray Bottle Trigger,Clam Nectar,Bandage - Flexible Neon
4,412,"Turnip - White, Organic","Beef - Tenderlion, Center Cut",Pastry - Butterscotch Baked,Bandage - Flexible Neon,Squid U5 - Thailand
...,...,...,...,...,...,...
995,97928,"Garlic - Primerba, Paste",Spinach - Baby,Cornflakes,"Cheese - Brie,danish",Sea Bass - Whole
996,98069,Cheese - Taleggio D.o.p.,Bandage - Flexible Neon,Juice - V8 Splash,Soup Campbells - Italian Wedding,Bread Crumbs - Panko
997,98159,"Coconut - Shredded, Sweet",Clam Nectar,"Turnip - White, Organic",Pastry - Butterscotch Baked,Pastry - Raisin Muffin - Mini
998,98185,Pecan Raisin - Tarts,Mustard Prepared,Chinese Foods - Chicken,Soup V8 Roasted Red Pepper,Tray - 16in Rnd Blk


In [ ]:
### Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [42]:

distance_metric = 'cosine'  

recommendations_dict = {}

for customer_id in unique_customer_ids:
    
    distance_matrix = pdist(product_customer_matrix.T, metric=distance_metric)
    similarity_matrix = pd.DataFrame(squareform(distance_matrix), index=product_customer_matrix.columns, columns=product_customer_matrix.columns)

    
    similarities = similarity_matrix[customer_id].sort_values(ascending=False).index[1:]
    top_similar_customers = similarities[:5]
    selected_records = data_quantity[data_quantity['CustomerID'].isin(top_similar_customers)]
    aggregated_records = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
    ranked_records = aggregated_records.sort_values(by='Quantity', ascending=False)
    customer_purchases = product_customer_matrix[customer_id]
    merged_data = pd.merge(ranked_records, product_customer_matrix, on='ProductName')
    not_purchased_products = merged_data[merged_data[customer_id] == 0]
    top_recommendations = not_purchased_products.head(5)

    
    recommendations_dict[customer_id] = top_recommendations[['ProductName', 'Quantity']]


for customer_id, recommendations in recommendations_dict.items():
    print(f"CustomerID: {customer_id}\nRecommendations:\n{recommendations}\n{'='*40}\n")


CustomerID: 33
Recommendations:
               ProductName  Quantity
0     Cinnamon Buns Sticky        57
1        Juice - V8 Splash        55
2   Longos - Chicken Wings        55
3  Table Cloth 54x72 White        54
4  Tea - English Breakfast        54

CustomerID: 200
Recommendations:
                ProductName  Quantity
0    Pepper - White, Ground        58
1  Wine - White Cab Sauv.on        56
2             Peas - Frozen        56
3       Arizona - Green Tea        51
4        Beef - Rib Eye Aaa        45

CustomerID: 264
Recommendations:
                     ProductName  Quantity
0  Beer - Original Organic Lager        55
1    Pork - Bacon, Double Smoked        50
2           Lime Cordial - Roses        48
3     Wine - Magnotta - Belpaese        44
4              Lettuce - Treviso        38

CustomerID: 356
Recommendations:
                     ProductName  Quantity
0              Snapple Lemon Tea        63
1              Pail For Lid 1537        39
2           Chocolate - Feath

CustomerID: 32373
Recommendations:
             ProductName  Quantity
0  Hickory Smoke, Liquid        68
1                   Kiwi        68
2       Lettuce - Frisee        67
3    Lamb - Whole, Fresh        66
4     Cookies - Assorted        57

CustomerID: 32386
Recommendations:
                   ProductName  Quantity
0         Bread - Bistro White        58
1                      Campari        48
2           Cheese - Camembert        45
3                Sage - Ground        43
4  Cake - Box Window 10x10x2.5        40

CustomerID: 32558
Recommendations:
                  ProductName  Quantity
0  Lemonade - Natural, 591 Ml        91
1    Seedlings - Mix, Organic        90
2        Bread Foccacia Whole        71
3    Wine - White, Mosel Gold        71
4          Pasta - Angel Hair        70

CustomerID: 32575
Recommendations:
                       ProductName  Quantity
0         Seedlings - Mix, Organic        50
1       Wine - Two Oceans Cabernet        39
2                         

CustomerID: 59925
Recommendations:
                     ProductName  Quantity
0      Macaroons - Two Bite Choc        94
1     Table Cloth - 53x69 Colour        82
2  Island Oasis - Mango Daiquiri        74
3              Beef - Short Loin        68
4     Wine - Crozes Hermitage E.        64

CustomerID: 60185
Recommendations:
              ProductName  Quantity
0                Sardines        63
1     Squid U5 - Thailand        60
2      Peas - Pigeon, Dry        54
3  Wine - Ruffino Chianti        47
4    Cocktail Napkin Blue        45

CustomerID: 60378
Recommendations:
                        ProductName  Quantity
0  Pork - Back, Short Cut, Boneless       108
1                       Tofu - Firm        70
2                      Grapes - Red        69
3    French Pastry - Mini Chocolate        65
4        Soup - Campbells, Cream Of        53

CustomerID: 60389
Recommendations:
                ProductName  Quantity
0                 Milk - 2%        64
1        Onions - Cippolini    

CustomerID: 90054
Recommendations:
                  ProductName  Quantity
0           Tilapia - Fillets        77
1                  Watercress        56
2               Vanilla Beans        52
3                      Loquat        51
4  Wine - Two Oceans Cabernet        51

CustomerID: 90069
Recommendations:
                     ProductName  Quantity
0           Lamb - Pieces, Diced        53
2        Wine - Chardonnay South        43
3  Ocean Spray - Kiwi Strawberry        42
4             Tea - Decaf Lipton        42
5              Assorted Desserts        42

CustomerID: 90174
Recommendations:
                 ProductName  Quantity
0       Cinnamon Buns Sticky        65
1         Cheese - Cambozola        57
2    Table Cloth 54x72 White        57
3   Water - Aquafina Vitamin        57
4  Placemat - Scallop, White        53

CustomerID: 90193
Recommendations:
                     ProductName  Quantity
0     Turkey - Oven Roast Breast        79
1             Tea - Decaf Lipton       